Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

#### Regresion

In [11]:
batch_size = 128
regulize = .005

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(
                tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset,weights)+biases
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
    regulizer = regulize*(tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss+regulizer)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset,weights)+biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+biases)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step*batch_size) % (train_labels.shape[0]-batch_size)
        
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction],
                                        feed_dict = feed_dict)
        if(step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step,l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions,batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(),
                                                           valid_labels))
            print("Loss")
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(),test_labels))

Initialized
Minibatch loss at step 0: 17.009966
Minibatch accuracy: 9.4%
Validation accuracy: 14.1%
Loss
Minibatch loss at step 500: 0.713381
Minibatch accuracy: 80.5%
Validation accuracy: 79.8%
Loss
Minibatch loss at step 1000: 0.827555
Minibatch accuracy: 80.5%
Validation accuracy: 81.2%
Loss
Minibatch loss at step 1500: 0.826380
Minibatch accuracy: 83.6%
Validation accuracy: 81.9%
Loss
Minibatch loss at step 2000: 0.482933
Minibatch accuracy: 85.9%
Validation accuracy: 82.3%
Loss
Minibatch loss at step 2500: 0.741282
Minibatch accuracy: 80.5%
Validation accuracy: 81.8%
Loss
Minibatch loss at step 3000: 0.736153
Minibatch accuracy: 82.0%
Validation accuracy: 81.9%
Loss
Test accuracy: 88.3%


#### NN

In [17]:
batch_size = 128
regulize = .005

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, image_size*image_size]))
    biases = tf.Variable(tf.zeros([image_size*image_size]))
    weights2 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases),
                       weights2)+biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    regulizer = regulize*(tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases)+
                         tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases2))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss+regulizer)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases),weights2)+biases2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases),weights2)+biases2)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 293.090942
Minibatch accuracy: 6.2%
Validation accuracy: 31.7%
Minibatch loss at step 500: 1.192135
Minibatch accuracy: 79.7%
Validation accuracy: 81.7%
Minibatch loss at step 1000: 0.551909
Minibatch accuracy: 85.2%
Validation accuracy: 84.8%
Minibatch loss at step 1500: 0.599606
Minibatch accuracy: 85.2%
Validation accuracy: 85.4%
Minibatch loss at step 2000: 0.374867
Minibatch accuracy: 88.3%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 0.581889
Minibatch accuracy: 83.6%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 0.569982
Minibatch accuracy: 83.6%
Validation accuracy: 85.5%
Test accuracy: 91.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [18]:
num_steps = 2

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 319.125092
Minibatch accuracy: 4.7%
Validation accuracy: 37.4%
Test accuracy: 27.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

It doubles the accuracy!

In [19]:
batch_size = 128
regulize = .005
dropout = .5

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, image_size*image_size]))
    biases = tf.Variable(tf.zeros([image_size*image_size]))
    weights2 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
  
    # Training computation.
    #logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases),dropout),
     #                  weights2)+biases2
    
    
    #else:
     #   logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases),
                       #weights2)+biases2
    def get_Logits(data,train=True):
        if train:
            logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(data, weights) + biases),dropout),
                       weights2)+biases2
        else:
            logits = tf.matmul(tf.nn.relu(tf.matmul(data, weights) + biases),
                       weights2)+biases2
        return(logits)
    logits = get_Logits(tf_train_dataset,train=True)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    regulizer = regulize*(tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases)+
                         tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases2))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss+regulizer)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(get_Logits(tf_valid_dataset,False))
    #valid_prediction = tf.nn.softmax(
     #   tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases),weights2)+biases2)
    test_prediction = tf.nn.softmax(get_Logits(tf_test_dataset,False))
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases),weights2)+biases2)

In [20]:
num_steps = 2

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 480.818512
Minibatch accuracy: 8.6%
Validation accuracy: 26.6%
Test accuracy: 45.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [21]:
batch_size = 128
regulize = .005
dropout = .5

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, image_size*image_size]))
    biases = tf.Variable(tf.zeros([image_size*image_size]))
    weights2 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
  
    # Training computation.

    def get_Logits(data,train=True):
        if train:
            logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(data, weights) + biases),dropout),
                       weights2)+biases2
        else:
            logits = tf.matmul(tf.nn.relu(tf.matmul(data, weights) + biases),
                       weights2)+biases2
        return(logits)
    logits = get_Logits(tf_train_dataset,train=True)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    regulizer = regulize*(tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases)+
                         tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases2))
  
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step,500,0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss+regulizer, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(get_Logits(tf_valid_dataset,False))
    test_prediction = tf.nn.softmax(get_Logits(tf_test_dataset,False))
    

In [22]:
num_steps = 200001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions,_ = session.run(
            [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 369.696411
Minibatch accuracy: 10.2%
Validation accuracy: 25.9%
Minibatch loss at step 500: 1.715115
Minibatch accuracy: 78.9%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 0.679626
Minibatch accuracy: 83.6%
Validation accuracy: 84.3%
Minibatch loss at step 1500: 0.663911
Minibatch accuracy: 80.5%
Validation accuracy: 84.8%
Minibatch loss at step 2000: 0.422667
Minibatch accuracy: 86.7%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 0.598162
Minibatch accuracy: 82.0%
Validation accuracy: 84.5%
Minibatch loss at step 3000: 0.607233
Minibatch accuracy: 84.4%
Validation accuracy: 85.0%
Minibatch loss at step 3500: 0.586733
Minibatch accuracy: 82.0%
Validation accuracy: 85.3%
Minibatch loss at step 4000: 0.620839
Minibatch accuracy: 84.4%
Validation accuracy: 85.2%
Minibatch loss at step 4500: 0.532963
Minibatch accuracy: 80.5%
Validation accuracy: 84.5%
Minibatch loss at step 5000: 0.509288
Minibatch accuracy: 84.4%
Validation accur

### sklearn attempt

In [24]:
from sklearn.neural_network import MLPClassifier
NN=MLPClassifier(algorithm='adam',verbose=True,hidden_layer_sizes=(image_size*image_size))
size=150000
NN.fit(train_dataset[0:size],train_labels[0:size])
pred_NN_train = NN.predict(train_dataset[0:size])
pred_NN_test = NN.predict(test_dataset)
from sklearn.metrics import accuracy_score
training_error_NN = 1 - accuracy_score(train_labels[0:size],pred_NN_train,normalize=True)
print(training_error_NN)
test_error_NN = 1 - accuracy_score(test_labels,pred_NN_test,normalize=True)
print(test_error_NN)

Iteration 1, loss = 0.86278292
Iteration 2, loss = 0.65438487
Iteration 3, loss = 0.58246130
Iteration 4, loss = 0.53282412
Iteration 5, loss = 0.49288043
Iteration 6, loss = 0.45529937
Iteration 7, loss = 0.42354453
Iteration 8, loss = 0.39359076
Iteration 9, loss = 0.36541348
Iteration 10, loss = 0.34132595
Iteration 11, loss = 0.31693050
Iteration 12, loss = 0.29688018
Iteration 13, loss = 0.27913592
Iteration 14, loss = 0.26044290
Iteration 15, loss = 0.24534547
Iteration 16, loss = 0.23105979
Iteration 17, loss = 0.21414010
Iteration 18, loss = 0.20638913
Iteration 19, loss = 0.19472250
Iteration 20, loss = 0.18378902
Iteration 21, loss = 0.17577533
Iteration 22, loss = 0.16862494
Iteration 23, loss = 0.16191637
Iteration 24, loss = 0.15664395
Iteration 25, loss = 0.14822793
Iteration 26, loss = 0.14549493
Iteration 27, loss = 0.13935677
Iteration 28, loss = 0.13284767
Iteration 29, loss = 0.13138203
Iteration 30, loss = 0.12772055
Iteration 31, loss = 0.12351373
Iteration 32, los

They're the same!